<a href="https://colab.research.google.com/github/comojin1994/ResNet/blob/master/Exercise_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
tf.__version__

TensorFlow 2.x selected.


'2.1.0'

### Hyperparameter

In [0]:
EPOCHS = 10
batch_size = 32

### ResidualUnit

In [0]:
class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')

        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1, 1), padding='same')

    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        return self.identity(x) + h

### ResidualLayer

In [0]:
class ResidualLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResidualLayer, self).__init__()
        self.sequence = list()
        for f_in, f_out in zip([filter_in] + list(filters), filters):
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))        

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x

### Model

In [0]:
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(8, (3, 3), padding='same', activation='relu') # 32x32x8

        self.res1 = ResidualLayer(8, (16, 16), (3, 3)) # 32x32x16
        self.pool1 = tf.keras.layers.MaxPool2D() # 16x16x16

        self.res2 = ResidualLayer(16, (32, 32), (3, 3)) # 16x16x32
        self.pool2 = tf.keras.layers.MaxPool2D() # 8x8x32

        self.res3 = ResidualLayer(32, (64, 64), (3, 3)) # 8x8x64
        self.pool3 = tf.keras.layers.MaxPool2D() # 4x4x64

        self.res4 = ResidualLayer(64, (128, 128), (3, 3)) # 4x4x128

        self.flatten = tf.keras.layers.Flatten() # 2048
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        x = self.res1(x, training=training)
        x = self.pool1(x)
        x = self.res2(x, training=training)
        x = self.pool2(x)
        x = self.res3(x, training=training)
        x = self.pool3(x)
        x = self.res4(x, training=training)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

### Load Data

In [0]:
cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255., x_test / 255.
x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

170500096/170498071 [==============================] - 13s 0us/step


### Define train, test loop

In [0]:
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

### Define model, loss function, optimizer and metric

In [0]:
model = ResNet()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Training

In [0]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for images, labels in test_ds:
        test_step(model, images, labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(
          epoch + 1,
          train_loss.result(),
          train_accuracy.result() * 100,
          test_loss.result(),
          test_accuracy.result() * 100
    ))

Epoch 1, Loss: 1.4326711893081665, Accuracy50.19000244140625, Test Loss: 1.1125925779342651, Test Accuracy: 60.93000030517578
Epoch 2, Loss: 1.1854692697525024, Accuracy58.6619987487793, Test Loss: 1.062851905822754, Test Accuracy: 63.195003509521484
Epoch 3, Loss: 1.0400160551071167, Accuracy63.77666473388672, Test Loss: 0.9904375076293945, Test Accuracy: 65.72666931152344
Epoch 4, Loss: 0.9387577772140503, Accuracy67.385498046875, Test Loss: 0.9522490501403809, Test Accuracy: 67.20249938964844
Epoch 5, Loss: 0.8582229018211365, Accuracy70.23040008544922, Test Loss: 0.9284880757331848, Test Accuracy: 68.31200408935547
Epoch 6, Loss: 0.7903414368629456, Accuracy72.62266540527344, Test Loss: 0.9162169098854065, Test Accuracy: 69.36333465576172
Epoch 7, Loss: 0.7334502339363098, Accuracy74.59600067138672, Test Loss: 0.8926169872283936, Test Accuracy: 70.47856903076172
Epoch 8, Loss: 0.6832081079483032, Accuracy76.34574890136719, Test Loss: 0.9010035991668701, Test Accuracy: 70.7512512207

In [0]:
model.summary()

Model: "res_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           multiple                  224       
_________________________________________________________________
residual_layer_8 (ResidualLa multiple                  8496      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 multiple                  0         
_________________________________________________________________
residual_layer_9 (ResidualLa multiple                  33376     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 multiple                  0         
_________________________________________________________________
residual_layer_10 (ResidualL multiple                  132288    
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 multiple                  0 

In [0]:
model.compile(optimizer=optimizer, loss=loss_object, metrics=['accuracy'])
model.evaluate(test_ds)

313/313 [==============================] - 2s 6ms/step - loss: 0.9931 - accuracy: 0.7474


[0.9930694424591887, 0.7474]